# Ironbox: Enhanced Agent Core and Framework Selection

This notebook demonstrates how the enhanced Agent Core processes queries and selects the appropriate framework for handling different types of requests.

## Overview

The Agent Core is the central component of the Ironbox system that:
1. Receives user queries
2. Uses the Framework Selector to determine the best framework for handling the query
3. Routes the query to the selected framework
4. Returns the response to the user

The Framework Selector can choose from four frameworks:
- **Route Framework**: For simple categorizable queries that can be handled by specialized agents
- **React Framework**: For reasoning and action problems that require tool use
- **Plan Framework**: For complex multi-step problems that require planning
- **Direct LLM**: For simple informational questions that can be answered directly

The enhanced architecture introduces a **Unified Toolkit** that allows all frameworks to access both tools and agents, providing greater flexibility and power.

Let's explore how this works in practice.

## Setup

First, let's import the necessary modules and initialize the Agent Core.

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import Ironbox components
from ironbox.core.agent_core import AgentCore
from ironbox.core.agent_framework import AgentFramework, RouteAgentFramework, ReactAgentFramework, PlanAgentFramework
from ironbox.config import load_config

# Load configuration
config = load_config()

# Initialize the Agent Core with configuration
agent_core = AgentCore(config=config)

# Initialize the Agent Core (this will set up the toolkit and frameworks)
await agent_core.initialize()

# Setup is complete
print("Enhanced Agent Core initialized successfully.")

## Unified Toolkit

The enhanced architecture introduces a Unified Toolkit that manages all tools and agents. Let's examine what's available in the toolkit.

In [ ]:
# List all tools in the toolkit
print("All Tools in Unified Toolkit:")
for tool_name in agent_core.toolkit.tools.keys():
    print(f"- {tool_name}")

print("\nLocal Tools:")
for tool_name in agent_core.toolkit.local_tools.keys():
    print(f"- {tool_name}")

print("\nMCP Tools:")
for tool_name in agent_core.toolkit.mcp_tools.keys():
    print(f"- {tool_name}")

print("\nAgent-as-Tools:")
for tool_name in agent_core.toolkit.agent_tools.keys():
    print(f"- {tool_name}")

print("\nOriginal Agents:")
for agent_name in agent_core.toolkit.agents.keys():
    print(f"- {agent_name}")

## Framework Selection

Let's examine how the Framework Selector decides which framework to use for different types of queries.

In [ ]:
# Function to demonstrate framework selection
async def demonstrate_framework_selection(query):
    print(f"Query: {query}")
    framework_type = await agent_core.framework_selector.select_framework(query)
    print(f"Selected Framework: {framework_type}\n")
    return framework_type

# Test with different types of queries
queries = [
    "Register my Kubernetes cluster at endpoint https://k8s.example.com",  # Route Framework (Cluster Register)
    "How many pods are running in my production namespace?",               # React Framework (requires tool use)
    "Create a deployment plan for migrating my application to Kubernetes", # Plan Framework (complex multi-step)
    "What is Kubernetes?"                                                 # Direct LLM (informational)
]

for query in queries:
    await demonstrate_framework_selection(query)

## Processing Queries with Different Frameworks

Now let's see how each framework processes a query.

In [ ]:
# Function to process a query using the Agent Core
async def process_query(query):
    print(f"Processing query: {query}")
    response = await agent_core.process_query(query, session_id="demo-session")
    print(f"Response: {response['response']}")
    print(f"Framework used: {response['framework']}\n")
    return response

# Process each query
for query in queries:
    await process_query(query)

## Route Framework in Detail

Let's take a closer look at how the Route Framework works with specialized agents.

In [ ]:
# Get the Route Framework
route_framework = agent_core.frameworks.get("route")

# List available specialized agents
print("Available Specialized Agents:")
for agent_type in agent_core.toolkit.agents.keys():
    print(f"- {agent_type}")

# Process a query with the Route Framework
route_query = "Check the health of my production cluster"
print(f"\nProcessing query with Route Framework: {route_query}")
from ironbox.core.graph import AgentState
state = AgentState(input=route_query, session_id="demo-session")
result_state = await route_framework.process(state)
print(f"Response: {result_state.response if hasattr(result_state, 'response') else 'No direct response'}")
if hasattr(result_state, 'agent_outputs') and 'router' in result_state.agent_outputs:
    print(f"Next agent: {result_state.agent_outputs['router'].get('next', 'None')}")

## React Framework in Detail

Now let's examine how the React Framework uses tools to solve problems.

In [ ]:
# Get the React Framework
react_framework = agent_core.frameworks.get("react")

# List available tools
print("Available Tools for React Framework:")
for tool_name in agent_core.toolkit.tools.keys():
    print(f"- {tool_name}")

# Process a query with the React Framework
react_query = "Scale my frontend deployment to 3 replicas"
print(f"\nProcessing query with React Framework: {react_query}")
state = AgentState(input=react_query, session_id="demo-session")
result_state = await react_framework.process(state)
print(f"Response: {result_state.response if hasattr(result_state, 'response') else 'No direct response'}")

# Print steps if available
if hasattr(result_state, 'agent_outputs') and 'react' in result_state.agent_outputs:
    steps = result_state.agent_outputs['react'].get('steps', [])
    if steps:
        print("\nThinking steps:")
        for i, step in enumerate(steps):
            print(f"Step {i+1}:")
            print(f"  Thought: {step.get('thought', 'No thought')}")
            print(f"  Action: {step.get('action', 'No action')}")
            print(f"  Action Input: {step.get('action_input', 'No input')}")
            print(f"  Observation: {step.get('observation', 'No observation')}\n")

## Plan Framework in Detail

Now let's see how the Plan Framework creates and executes plans for complex problems.

In [ ]:
# Get the Plan Framework
plan_framework = agent_core.frameworks.get("plan")

# Process a query with the Plan Framework
plan_query = "Create a new namespace, deploy a Redis instance, and configure it for high availability"
print(f"Processing query with Plan Framework: {plan_query}")
state = AgentState(input=plan_query, session_id="demo-session")
result_state = await plan_framework.process(state)
print(f"Response: {result_state.response if hasattr(result_state, 'response') else 'No direct response'}")

# Display the plan and execution results
if hasattr(result_state, 'agent_outputs') and 'plan' in result_state.agent_outputs:
    plan = result_state.agent_outputs['plan'].get('plan', [])
    if plan:
        print("\nPlan:")
        for i, step in enumerate(plan):
            status = "✓" if step.completed else "✗"
            print(f"{status} Step {i+1}: {step.description}")
            if step.result:
                print(f"   Result: {step.result}")

## Using Agents as Tools

One of the key enhancements in the new architecture is the ability to use agents as tools. Let's see how this works.

In [ ]:
# List agent tools
print("Agent Tools:")
agent_tools = [name for name in agent_core.toolkit.tools.keys() if name.startswith("agent_")]
for tool_name in agent_tools:
    print(f"- {tool_name}")

# Process a query that would use an agent as a tool
agent_tool_query = "Use the cluster health agent to check the health of the production cluster"
print(f"\nProcessing query that uses an agent as a tool: {agent_tool_query}")
response = await agent_core.process_query(agent_tool_query, session_id="demo-session")
print(f"Response: {response['response']}")
print(f"Framework used: {response['framework']}")

# Print steps if available and using React framework
if response['framework'] == 'react' and 'state' in response:
    state = response['state']
    if hasattr(state, 'agent_outputs') and 'react' in state.agent_outputs:
        steps = state.agent_outputs['react'].get('steps', [])
        if steps:
            print("\nSteps:")
            for i, step in enumerate(steps):
                print(f"Step {i+1}:")
                print(f"  Thought: {step.get('thought', 'No thought')}")
                print(f"  Action: {step.get('action', 'No action')}")
                print(f"  Action Input: {step.get('action_input', 'No input')}")
                print(f"  Observation: {step.get('observation', 'No observation')}")

## Configuration-Based Setup

Another key enhancement is the ability to configure tools and agents through configuration files. Let's see what's in our configuration.

In [ ]:
# Display toolkit configuration
import json
print("Toolkit Configuration:")
print(json.dumps(config.get("toolkit", {}), indent=2))

## Conclusion

In this notebook, we've explored how the Enhanced Agent Core and Framework Selection system works in Ironbox:

1. The Agent Core receives user queries and uses the Framework Selector to determine the best framework for handling them.
2. The Unified Toolkit provides a central repository for all tools and agents, allowing any framework to access them.
3. The Route Framework routes simple categorizable queries to specialized agents.
4. The React Framework uses a thinking-action-observation loop to solve reasoning and action problems.
5. The Plan Framework creates and executes plans for complex multi-step problems.
6. Direct LLM responses handle simple informational questions.
7. Agents can be used as tools, allowing React and Plan frameworks to leverage specialized agent capabilities.
8. Tools and agents can be configured through configuration files, making the system more flexible and extensible.

This enhanced architecture allows Ironbox to handle a wide range of query types efficiently, using the most appropriate approach for each situation, while providing greater flexibility and power through the unified toolkit.